In [ ]:
import crossref_commons.retrieval

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
articles = pd.read_json('article_data.json', lines=True)

In [ ]:
articles

In [ ]:
#import name csv table
names = pd.read_csv('wgnd_noctry.csv')  

In [ ]:
names

In [ ]:
#drop names where gender is unidentified
names = names[names['gender']!='?']

# DATA CLEANING

Siin on idee tekitada dataframed, mis vastavad juba dimension tabelitele (ilma primary key-ta), siis oleks hea nad parquet failidena lugeda parquet faili välja ning seejärel lugeda teises funktsioonis parquet failid sisse ja sisestada tabelitesse korraga. See peaks olema kiirem ja lahendama probleemi, et funktsioonide vahel ei saa andmeid jagada dag-is.

In [ ]:
#drop all values where doi is null
articles = articles[articles['doi'].notna()]
#drop all values where name is null
articles = articles[articles['authors'].notna()]

In [ ]:
articles

In [ ]:
#drop abstract, report-no and license
articles = articles.drop(columns=['report-no', 'abstract','license','submitter'],axis=1)

In [ ]:
articles = articles.fillna(value=np.nan)
articles['doi'] = articles['doi'].astype('str') 

In [ ]:
articles["pages"] = articles["comments"].str.findall("\d+ page")
articles["pages"] = articles["pages"].replace(np.nan)
articles['pages'] = articles['pages'].astype('str') 
articles["pages"] = articles["pages"].replace("[]","0")
articles["pages"] = articles["pages"].str.extract("(\d+)")

In [ ]:
articles["figures"] = articles["comments"].str.findall("\d+ figure")
articles["figures"] = articles["figures"].replace(np.nan)
articles['figures'] = articles['figures'].astype('str') 
articles["figures"] = articles["figures"].replace("[]","0")
articles["figures"] = articles["figures"].str.extract("(\d+)")

In [ ]:
articles = articles.drop(columns=['comments'],axis=1)

In [ ]:
articles['id'] = articles['id'].astype('str') 
articles["year"] = 2000 + articles["id"].str[0].astype(int)
articles["month"] = articles["id"].str[1:3].astype(int)

In [ ]:
#version table
version = articles[["id","doi","versions"]]
authors = articles[["id","doi","authors","authors_parsed"]]
categories = articles[["id","doi","categories"]]
#columns to drop-versions, authors,authors_parsed

In [ ]:
#TODO
#create dimension table dataframes
#must parse author table with names, versions table,


## DATA ENRICHING

In [ ]:
#näide crossref api kasutamisest, et saada viiteid
d = crossref_commons.retrieval.get_publication_as_json('10.1103/PhysRevD.76.013009')
df = pd.json_normalize(d, 'reference')

notYetIncluded = df[~df["DOI"].astype(str).isin(articles["doi"].astype(str))]

In [ ]:
#select n articles from category, if article with same doi available save doi1-doi2
#töötab põhimõttel, et kasutab vaid olemasolevaid artikleid
find_citations = articles.groupby('categories').head(1).reset_index(drop=True)
find_citations = find_citations[["id","doi"]]
find_citations

In [ ]:
#VÄGA AEGLANE!!! (ja teeb midagi ikka valesti)
final_citations = pd.DataFrame({'doi_1' : [],'doi_2' : []})
doi_array = find_citations["doi"]
#i=0
for doi in doi_array:
    try:
        d = crossref_commons.retrieval.get_publication_as_json(doi)
        df = pd.json_normalize(d, 'reference')
        df = df[~df["DOI"].astype(str).isin(articles["doi"].astype(str))]
        df["doi_2"] = df["DOI"]
        df["doi_1"] = doi
        df = df[["doi_1","doi_2"]]
        final_citations = pd.concat([final_citations,df])
        
        #print(i)
        #i+=1
    except:
        #except when api request with doi does not exist
        #see pole best practice, aga atm teeb töö ära 
        pass

In [ ]:
final_citations

In [ ]:
df.to_csv('citations.csv') 